In [11]:
import boto3
from dotenv import load_dotenv
import os
load_dotenv()

True

In [24]:
def create_red_shift():
    redshift = boto3.client('redshift',
                        region_name='us-east-1',
                        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY"), 
                        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESSKEY"))
    return redshift

redshift = create_red_shift()
response = redshift.create_cluster(
    ClusterIdentifier='stock-project-cluster',
    NodeType=os.environ.get("NODE_TYPE"),
    ClusterType='single-node',
    DBName=os.environ.get("DB_NAME"),
    MasterUsername=os.environ.get("REDSHIFT_USER"),
    MasterUserPassword=os.environ.get("REDSHIFT_PASS"),
)
print(red_shift)